# Text to Image using Stability AI model

We can create images from text prompt using a number of models available in Amazon bedrock. Important inference parameters to consider while generating image from a text are,
- **text prompt** - Can be positive and negative
- **seed** - Random seed for reproducibility
- **size** - Height and Width
- **cfg-scale** - Indicates how close the image should for the prompt. Lower number will increase randomness in the generation of the image.

Note - To find the exact parameter names, refer to model documentation.

We are going to generate an image of a flower `rose` using Stability AI model - `stability.stable-diffusion-xl-v1`

Documentaion - https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html 

## Step 1: Configure AWS environment

In [1]:
# Set env variables using os module .
## NOT RECOMMENDED FOR PRODUCTION DEPLOYMENTS OR TO SHARE ON PUBLIC DOMAIN
import os
os.environ['AWS_ACCESS_KEY_ID'] = 'your key id'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'your secret'

os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'

## Step 2: Explore and choose the model of your choice
We will use Stability AI model - `stability.stable-diffusion-xl-v1`

In [2]:
# Cretae boto3 client to access bedrock APIs
import boto3

In [3]:
bedrock = boto3.client(service_name='bedrock',
                      region_name='us-east-1')

In [4]:
# To list available models
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': '314391ff-6af4-4bbc-9bc5-58b4b672ee55',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 May 2025 05:11:23 GMT',
   'content-type': 'application/json',
   'content-length': '54644',
   'connection': 'keep-alive',
   'x-amzn-requestid': '314391ff-6af4-4bbc-9bc5-58b4b672ee55'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

In [5]:
# Get model details
bedrock.get_foundation_model(modelIdentifier='stability.stable-diffusion-xl-v1')

{'ResponseMetadata': {'RequestId': '1871493a-dec1-42f2-a238-a6c473d415ab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 09 May 2025 05:11:23 GMT',
   'content-type': 'application/json',
   'content-length': '532',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1871493a-dec1-42f2-a238-a6c473d415ab'},
  'RetryAttempts': 0},
 'modelDetails': {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/stability.stable-diffusion-xl-v1',
  'modelId': 'stability.stable-diffusion-xl-v1',
  'modelName': 'SDXL 1.0',
  'providerName': 'Stability AI',
  'inputModalities': ['TEXT', 'IMAGE'],
  'outputModalities': ['IMAGE'],
  'customizationsSupported': [],
  'inferenceTypesSupported': ['ON_DEMAND'],
  'modelLifecycle': {'status': 'LEGACY'}}}

## Step 3: Prepare prompt for the model

As per the documentation below is the request format,
```
{
        "text_prompts": [
            {
                "text": string,
                "weight": float
            }
        ],
        "height": int,
        "width": int,
        "cfg_scale": float,
        "clip_guidance_preset": string,
        "sampler": string,
        "samples",
        "seed": int,
        "steps": int,
        "style_preset": string,
        "extras" :JSON object
        
}
```
We will use only few parameters in this example

In [6]:
# Use json module to construct the requets payload
import json

In [7]:
# Define the prompt for the model.
positive_prompt = "Create a picture of a natural rose flower with dews on the pettles, leaf and stem"
negative_prompt = "paining, drawing, blury"

req = json.dumps({
        "text_prompts": [
            {
                "text": positive_prompt,
                "weight": 1
            },
            {
                "text": negative_prompt,
                "weight": -1
            },            
        ],
        "height": 1024,
        "width": 1024,
        "cfg_scale": 7,
        "samples": 1,
        "seed": 5,
        "steps": 30        
})
print(req)

{"text_prompts": [{"text": "Create a picture of a natural rose flower with dews on the pettles, leaf and stem", "weight": 1}, {"text": "paining, drawing, blury", "weight": -1}], "height": 1024, "width": 1024, "cfg_scale": 7, "samples": 1, "seed": 5, "steps": 30}


## Step 4: Create bedrock runtime to invoke the model

In [8]:
bedrock_runtime = boto3.client(service_name='bedrock-runtime', region_name='us-east-1')

## Step 5: Invoke the model to get the image generated

In [9]:
response = bedrock_runtime.invoke_model(body=req, modelId='stability.stable-diffusion-xl-v1')

## Step 6: Process the response to get the image
Usually the image will be in base64 format. Need to use base64 decoder to get the image in the formats such as png, jpeg etc.
Stability AI model response format is as below,
```
{
    "result": string,
    "artifacts": [
        {
            "seed": int,
            "base64": string,
            "finishReason": string
        }
    ]
}
```
Lets use base64 and PIL.Image models for this purpose

In [10]:
response_body = json.loads(response.get('body').read())
#response_body

In [11]:
# Use base64 module to decode the image
import base64

In [12]:
decoded_image = base64.b64decode(response_body['artifacts'][0]['base64'])
#decoded_image

In [13]:
# If you encounter PIL not found, then install Pillow - works on MacOS
!pip install Pillow

Looking in indexes: https://pypi.python.org/simple


In [14]:
# To export the image as png, lets use PIL.Image
from PIL import Image

In [15]:
with open("rose.png", 'wb') as file:
    file.write(decoded_image)
#Image.open("rose.png")